In [43]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
import ZebraLib as zb
from scipy.optimize import curve_fit

class cilindro():
    def __init__(self, d, k, c, L, p, h):
        self.d = d
        self.k = k
        self.c = c
        self.L = L
        self.p = p
        self.h = h

    @property
    def bi(self):
        r = self.d/2
        V = np.pi*r**2*self.L
        As = 2*np.pi*r*(self.L+r)
        Lc = V/As
        return self.h*Lc/self.k

    @property
    def tau(self):
        r = self.d/2
        V = np.pi*(r**2)*self.L
        As = 2*np.pi*r*(self.L+r)
        return (self.p*self.c*V)/(self.h*As)

def T(E):
    return 4.0 + 22.9*E

def dist_temp(t, tau, ti, t_inf):
    return ((np.e**(-t/tau))*(ti - t_inf)) + t_inf

### Plot dos resultados experimentais

In [55]:
#import dos dados
df = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. TCM I/Relatório_4/dados_brutos_mV.xlsx')

#Transformando os mv em C
df2 = pd.DataFrame({'ca1':T(df.ca1), 'cr1':T(df.cr1), 'ca2':T(df.ca2), 'cr2':T(df.cr2), 
                    'ca3':T(df.ca3), 'cr3':T(df.cr3), 'ca4':T(df.ca4), 'cr4':T(df.cr4)})
                    
df2

,ca1,cr1,ca2,cr2,ca3,cr3,ca4,cr4
0,25.8466,41.0064,26.2588,41.6018,26.9916,42.6094,26.8771,41.3957
1,28.6404,40.8919,34.6402,41.5102,30.3808,42.4949,28.6633,41.3957
2,30.7472,40.8232,36.5409,41.3957,33.1288,42.2201,30.2434,41.2583
3,33.4952,40.7087,38.2584,41.2125,35.1669,41.9453,31.7548,41.1667
4,33.9074,40.5942,39.0370,41.0751,36.7470,41.7163,33.0372,41.0522
...,...,...,...,...,...,...,...,...
104,40.7545,37.5485,42.9987,35.9684,42.5865,34.4112,41.2812,36.1516
105,40.7545,37.5485,42.9987,35.9684,42.5865,34.4112,41.2812,36.0829
106,40.7545,37.5485,42.9987,35.9684,42.5865,34.4112,41.2812,36.0829
107,40.7545,37.5485,42.9987,35.9684,42.5865,34.4112,41.2812,36.0142


In [ ]:
0 &	25,84 &	41,00 &	26,25 &	41,60 &	26,99 &	42,60 &	26,87 &	41,39 \\ \hline
1 &	28,64 &	40,89 &	34,64 &	41,51 &	30,38 &	42,49 &	28,66 &	41,39 \\ \hline
2 &	30,74 &	40,82 &	36,54 &	41,39 &	33,12 &	42,22 &	30,24 &	41,25 \\ \hline
3 &	33,49 &	40,70 &	38,25 &	41,21 &	35,16 &	41,94 &	31,75 &	41,16 \\ \hline
4 &	33,90 &	40,59 &	39,03 &	41,07 &	36,74 &	41,71 &	33,03 &	41,05 \\ \hline
...	& ...	&...&	...	&...	&...	&...	&...	&... \\ \hline
104 &	40,75 &	37,54 &	42,99 &	35,96 &	42,58 &	34,41 &	41,28 &	36,15 \\ \hline
105 &	40,75 &	37,54 &	42,99 &	35,96 &	42,58 &	34,41 &	41,28 &	36,08 \\ \hline
106 &	40,75 &	37,54 &	42,99 &	35,96 &	42,58 &	34,41 &	41,28 &	36,08 \\ \hline
107 &	40,75 &	37,54 &	42,99 &	35,96 &	42,58 &	34,41 &	41,28 &	36,01 \\ \hline
108 &	40,75 &	37,54 &	42,99 &	35,96 &	42,58 &	34,41 &	41,28 &	35,99 \\ \hline

### Cilindro 1

In [45]:
ca1 = cilindro(0.05, 237, 896, 0.200, 2707, 500)
cr1 = cilindro(0.05, 237, 896, 0.200, 2707, 20)

fig, ax = plt.subplots()

ax.set(xlim=(0, 500), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

t = np.linspace(0.1, 500)

ax.plot(df.tempo, T(df.ca1), label = "Aquecimento Experimental", c='deeppink', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, ca1.tau, 26, 40), c='deeppink', label = "Aquecimento Analitico")

ax.plot(df.tempo, T(df.cr1), label = "Resfriamento Experimental",  c='royalblue', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, cr1.tau, 40, 26), label = "Resfriamento Analitico", c='royalblue')

ax.grid(ls='dotted')
ax.legend()
fig.savefig('figura1.pdf', bbox_inches='tight')

### Cilindro 2

In [46]:
ca2 = cilindro(0.05, 237, 896, 0.085, 2707, 500)
cr2 = cilindro(0.05, 237, 896, 0.085, 2707, 20)

fig, ax = plt.subplots()

ax.set(xlim=(0, 500), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

t = np.linspace(0.1, 500)

ax.plot(df.tempo, T(df.ca2), label = "Aquecimento Experimental", c='deeppink', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, ca2.tau, 26, 40), c='deeppink', label = "Aquecimento Analitico")

ax.plot(df.tempo, T(df.cr2), label = "Resfriamento Experimental",  c='royalblue', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, cr2.tau, 40, 26), label = "Resfriamento Analitico", c='royalblue')

ax.grid(ls='dotted')
ax.legend()
fig.savefig('figura2.pdf', bbox_inches='tight')

### Cilindro 3

In [47]:
ca3 = cilindro(0.035, 237, 896, 0.200, 2707, 500)
cr3 = cilindro(0.035, 237, 896, 0.200, 2707, 20)

fig, ax = plt.subplots()

ax.set(xlim=(0, 500), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

t = np.linspace(0.1, 500)

ax.plot(df.tempo, T(df.ca2), label = "Aquecimento Experimental", c='deeppink', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, ca2.tau, 26, 40), c='deeppink', label = "Aquecimento Analitico")

ax.plot(df.tempo, T(df.cr2), label = "Resfriamento Experimental",  c='royalblue', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, cr2.tau, 40, 26), label = "Resfriamento Analitico", c='royalblue')

ax.grid(ls='dotted')
ax.legend()
fig.savefig('figura3.pdf', bbox_inches='tight')

### Cilindro 4

In [48]:
ca4 = cilindro(0.05, 413, 383, 0.200, 8954, 500)
cr4 = cilindro(0.05, 413, 383, 0.200, 8954, 20)

fig, ax = plt.subplots()

ax.set(xlim=(0, 500), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

t = np.linspace(0.1, 500)

ax.plot(df.tempo, T(df.ca4), label = "Aquecimento Experimental", c='deeppink', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, ca4.tau, 26, 40), c='deeppink', label = "Aquecimento Analitico")

ax.plot(df.tempo, T(df.cr4), label = "Resfriamento Experimental",  c='royalblue', marker='x', markersize=4, ls=':')
ax.plot(t, dist_temp(t, cr4.tau, 40, 26), label = "Resfriamento Analitico", c='royalblue')

ax.grid(ls='dotted')
ax.legend()
fig.savefig('figura4.pdf', bbox_inches='tight')

In [50]:
ca1 = cilindro(0.05, 237, 896, 0.200, 2707, 500)
ca2 = cilindro(0.05, 237, 896, 0.085, 2707, 500)
ca3 = cilindro(0.035, 237, 896, 0.200, 2707, 500)
ca4 = cilindro(0.05, 413, 383, 0.200, 8954, 500)

cr1 = cilindro(0.05, 237, 896, 0.200, 2707, 20)
cr2 = cilindro(0.05, 237, 896, 0.085, 2707, 20)
cr3 = cilindro(0.035, 237, 896, 0.200, 2707, 20)
cr4 = cilindro(0.05, 413, 383, 0.200, 8954, 20)

cr = [cr1, cr2, cr3, cr4]
ca = [ca1, ca2, ca3, ca4]

crd = [df.cr1, df.cr2, df.cr3, df.cr4]
cad = [df.ca1, df.ca2, df.ca3, df.ca4]

t = np.linspace(0.1, 500)

for i in range(len(cr)):

    fig, ax = plt.subplots()
    
    ax.set(xlim=(0, 500), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

    #DADOS AQUECIMENTO
    ax.plot(df.tempo, T(cad[i]), label = "Aquecimento Experimental", c='red', marker='o', markersize=4, ls=':')
    ax.errorbar(df.tempo, T(cad[i]), xerr=.01/(3)**0.5, yerr=0.5/(3)**0.5, fmt='o', markeredgecolor='black', 
        markerfacecolor='red', markersize=5, capsize=5, ecolor='red')

    #DADOS RESFRIAMENTO
    ax.plot(df.tempo,  T(crd[i]), label = "Resfriamento Experimental",  c='royalblue', marker='o', markersize=4, ls=':')
    ax.errorbar(df.tempo,  T(crd[i]), xerr=.01/(3)**0.5, yerr=0.5/(3)**0.5, fmt='o', markeredgecolor='black', 
        markerfacecolor='royalblue', markersize=5, capsize=5, ecolor='royalblue')

    #ANALITICO - RESFRIAMENTO E AQUECIMENTO
    ax.plot(t, dist_temp(t, cr[i].tau, 40, 26), label = "Resfriamento Analitico", c='royalblue')
    ax.plot(t, dist_temp(t, ca[i].tau, 26, 40), c='red', label = "Aquecimento Analitico")

    ax.grid(ls='dotted')
    ax.legend()
    fig.savefig(f'figura{i+1}.pdf', bbox_inches='tight')


### Todas as curvas de aquecimento

In [51]:
fig, ax = plt.subplots()

ax.set(xlim=(0, 400), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

t = np.linspace(0.1, 10)

ax.errorbar(df.tempo, T(df.ca1), xerr=.01/(3)**0.5, yerr=0.5/(3)**0.5, fmt='o', markeredgecolor='black', 
    markerfacecolor='royalblue', markersize=5, capsize=5, ecolor='royalblue', ls='--', label='Cilindro 1')
ax.errorbar(df.tempo, T(df.ca2), xerr=.01/(3)**0.5, yerr=0.5/(3)**0.5, fmt='o', markeredgecolor='black', 
    markerfacecolor='deeppink', markersize=5, capsize=5, ecolor='deeppink', ls='--', label='Cilindro 2')
ax.errorbar(df.tempo, T(df.ca3), xerr=.01/(3)**0.5, yerr=0.5/(3)**0.5, fmt='o', markeredgecolor='black', 
    markerfacecolor='red', markersize=5, capsize=5, ecolor='red', ls='--', label='Cilindro 3')
ax.errorbar(df.tempo, T(df.ca4), xerr=.01/(3)**0.5, yerr=0.5/(3)**0.5, fmt='o', markeredgecolor='black', 
    markerfacecolor='limegreen', markersize=5, capsize=5, ecolor='limegreen', ls='--', label='Cilindro 4')

ax.grid(ls='dotted')
ax.legend()

fig.savefig('aquecimento.pdf', bbox_inches='tight')

### Todas as curvas de resfriamento

In [52]:
fig, ax = plt.subplots()

ax.set(xlim=(0, 500), xlabel = r'tempo $t$', ylabel = r'$Temperatura [º]$')

t = np.linspace(0.1, 10)

ax.plot(df.tempo, T(df.cr1), label = "Resfriamento cilindro 1", marker='o', markersize=3.5, ls='--')
ax.plot(df.tempo, T(df.cr2), label = "Resfriamento cilindro 2", marker='o', markersize=3.5, ls='--')
ax.plot(df.tempo, T(df.cr3), label = "Resfriamento cilindro 3", marker='o', markersize=3.5, ls='--')
ax.plot(df.tempo, T(df.cr4), label = "Resfriamento cilindro 4", marker='o', markersize=3.5, ls='--')

ax.grid(ls='dotted')
ax.legend()

fig.savefig('resfriamento.pdf', bbox_inches='tight')

### Cálculo do numero de Bi

In [53]:
ca1 = cilindro(0.05, 237, 896, 0.200, 2707, 500)
ca2 = cilindro(0.05, 237, 896, 0.085, 2707, 500)
ca3 = cilindro(0.035, 237, 896, 0.200, 2707, 500)
ca4 = cilindro(0.05, 413, 383, 0.200, 8954, 500)

cr1 = cilindro(0.05, 237, 896, 0.200, 2707, 20)
cr2 = cilindro(0.05, 237, 896, 0.085, 2707, 20)
cr3 = cilindro(0.035, 237, 896, 0.200, 2707, 20)
cr4 = cilindro(0.05, 413, 383, 0.200, 8954, 20)

cr = [cr1.tau, cr2.tau, cr3.tau, cr4.tau]
ca = [ca1.tau, ca2.tau, ca3.tau, ca4.tau]

pd.DataFrame({'a':ca, 'b':cr})

,a,b
0,53.899378,1347.484444
1,46.855709,1171.392727
2,39.030584,975.764598
3,76.208489,1905.212222


### Gerando os dados analíticos

In [54]:
t = np.linspace(0.1, 500, 109)

A1 = dist_temp(t, ca1.tau, 40, 26)
A2 = dist_temp(t, ca2.tau, 40, 26)
A3 = dist_temp(t, ca3.tau, 40, 26)
A4 = dist_temp(t, ca4.tau, 40, 26)

C1 = dist_temp(t, cr1.tau, 26, 40)
C2 = dist_temp(t, cr2.tau, 26, 40)
C3 = dist_temp(t, cr3.tau, 26, 40)
C4 = dist_temp(t, cr4.tau, 26, 40)

cr = [C1, C2, C3, C4]
ca = [A1, A2, A3, A4]

df = pd.DataFrame({'A1':A1, 'A2':A2, 'A3':A3, 'A4':A4, 'C1':C1, 'C2':C2, 'C3':C3, 'C4':C4})
df
#df.to_excel('D:/UNESP/EngMec-UNESP/Lab. TCM I/Relatório_4/dados_analitico.xlsx')

,A1,A2,A3,A4,C1,C2,C3,C4
0,39.974050,39.970153,39.964177,39.981641,26.001039,26.001195,26.001435,26.000735
1,38.824088,38.656068,38.402568,39.157708,26.049044,26.056402,26.067682,26.034705
2,37.768759,37.465591,37.015593,38.382329,26.096884,26.111390,26.133616,26.068592
3,36.800277,36.387094,35.783723,37.652643,26.144560,26.166162,26.199238,26.102397
4,35.911493,35.410045,34.689613,36.965957,26.192073,26.220718,26.264549,26.136120
...,...,...,...,...,...,...,...,...
104,26.001847,26.000482,26.000061,26.025250,30.206334,30.718522,31.452718,29.126405
105,26.001695,26.000437,26.000055,26.023762,30.239919,30.755125,31.493168,29.152790
106,26.001556,26.000396,26.000048,26.022361,30.273388,30.791583,31.533426,29.179112
107,26.001428,26.000359,26.000043,26.021044,30.306742,30.827898,31.573493,29.205369


In [ ]:
	A1	A2	A3	A4	C1	C2	C3	C4
0	39,97	& 39,97	& 39,96	& 39,98	& 26,00	& 26,00	& 26,00	& 26,00 \\ \hline
1	38,82	& 38,65	& 38,40	& 39,15	& 26,04	& 26,05	& 26,06	& 26,03 \\ \hline
2	37,76	& 37,46	& 37,01	& 38,38	& 26,09	& 26,11	& 26,13	& 26,06 \\ \hline
3	36,80	& 36,38	& 35,78	& 37,65	& 26,14	& 26,16	& 26,19	& 26,10 \\ \hline
4	35,91	& 35,41	& 34,68	& 36,96	& 26,19	& 26,22	& 26,26	& 26,13 \\ \hline
...	...	...	...	...	...	...	...	... \\ \hline 
104	26,00	& 26,00	& 26,00	& 26,02	& 30,20	& 30,71	& 31,45	& 29,12 \\ \hline
105	26,00	& 26,00	& 26,00	& 26,02	& 30,23	& 30,75	& 31,49	& 29,15 \\ \hline
106	26,00	& 26,00	& 26,00	& 26,02	& 30,27	& 30,79	& 31,53	& 29,17 \\ \hline
107	26,00	& 26,00	& 26,00	& 26,02	& 30,30	& 30,82	& 31,57	& 29,20 \\ \hline
108	26,00	& 26,00	& 26,00	& 26,01	& 30,33	& 30,86	& 31,61	& 29,23 \\ \hline

In [ ]:
	A1	A2	A3	A4	C1	C2	C3	C4
0	& 39,97	& 39,97	& 39,96	& 39,98	& 26,00	& 26,00	& 26,00	& 26,00 \\ \hline
1	& 38,82	& 38,65	& 38,40	& 39,15	& 26,04	& 26,05	& 26,06	& 26,03 \\ \hline
2	& 37,76	& 37,46	& 37,01	& 38,38	& 26,09	& 26,11	& 26,13	& 26,06 \\ \hline
3	& 36,80	& 36,38	& 35,78	& 37,65	& 26,14	& 26,16	& 26,19	& 26,10 \\ \hline
4	& 35,91	& 35,41	& 34,68	& 36,96	& 26,19	& 26,22	& 26,26	& 26,13 \\ \hline
...	& ...	&...	&...	&...	&...	&...	&...	& ... \\ \hline 
104	& 26,00	& 26,00	& 26,00	& 26,02	& 30,20	& 30,71	& 31,45	& 29,12 \\ \hline
105	& 26,00	& 26,00	& 26,00	& 26,02	& 30,23	& 30,75	& 31,49	& 29,15 \\ \hline
106	& 26,00	& 26,00	& 26,00	& 26,02	& 30,27	& 30,79	& 31,53	& 29,17 \\ \hline
107	& 26,00	& 26,00	& 26,00	& 26,02	& 30,30	& 30,82	& 31,57	& 29,20 \\ \hline
108	& 26,00	& 26,00	& 26,00	& 26,01	& 30,33	& 30,86	& 31,61	& 29,23 \\ \hline